# Accessing US Census Databases via APIs: code
* * * * *


In [1]:
# Import required libraries
import requests
import urllib
import json
from __future__ import division
import math

## API GET Request code (the key is hidden) 


In [2]:
# Use the US census API

# set key
key="2dff129fcfb9e1af4827127edab7b719cc0436a7"

# set base url; 
# American Community Servey 5 year data 2014
base_url="http://api.census.gov/data/2014/acs5?"

# set response format
response_format=".json"

# Search parameters 


I needed to go through the IDs of all the variables in the ACS 5 years data to collect the relevant socio-economic indicators. The rough description of the each indicator will appear in the loop code below. As for the detailed descriptions of each ID, please refer to the following page: http://api.census.gov/data/2010/acs5/variables.html

In [6]:
# Set the key, search parameters and geography
# "for":"county:*","in":"state:*" means "for every county in every state"

search_params = {"key":key,
                 "get":"NAME,B01001_001E,B01001H_001E,B01001I_001E,B01001B_001E,B01002_001E,B07001_017E,B07001_049E,B07001_065E,B07401_049E,B07401_065E,B16010_015E,B16010_041E,B06001_004E,B06001_011E,B06001_013E,B06011_001E,B25003_001E,B25003_002E,B22005C_002E,B27011_002E,B27011_014E,B27011_025E,C24050_004E,C24050_002E,C24050_015E,ST,COUNTY",
                 "for":"county:*","in":"state:*"}    

# Request get
r = requests.get(base_url, params=search_params)

# Read the requested data in json format 
data=json.loads(r.text)

# As you see, the first of the data is the list of variables 
data[:2]


[['NAME',
  'B01001_001E',
  'B01001H_001E',
  'B01001I_001E',
  'B01001B_001E',
  'B01002_001E',
  'B07001_017E',
  'B07001_049E',
  'B07001_065E',
  'B07401_049E',
  'B07401_065E',
  'B16010_015E',
  'B16010_041E',
  'B06001_004E',
  'B06001_011E',
  'B06001_013E',
  'B06011_001E',
  'B25003_001E',
  'B25003_002E',
  'B22005C_002E',
  'B27011_002E',
  'B27011_014E',
  'B27011_025E',
  'C24050_004E',
  'C24050_002E',
  'C24050_015E',
  'ST',
  'COUNTY',
  'state',
  'county'],
 ['Autauga County, Alabama',
  '55136',
  '42033',
  '1391',
  '10257',
  '37.9',
  '46299',
  '3057',
  '2068',
  '3224',
  '1387',
  '12139',
  '7950',
  '4689',
  '4561',
  '37521',
  '26728',
  '20304',
  '15248',
  '28',
  '25952',
  '1953',
  '14077',
  '3194',
  '380',
  '7802',
  '01',
  '001',
  '01',
  '001']]

In [7]:
##  Create a loop to make it into a list of dictionary 

# Omit the variables for the loop
data1=data[1:]

# Create a vacant list to fill in
formatted = []

# Loop for each element of the data1
for i in range(len(data1)):
    
    # Create a void dictionary 
    dic={}
    
    # NAME: County, State (str)
    dic['CountyState'] = data1[i][0]
    
    # B01001~ : Population (int)
    dic['Population'] = data1[i][1]
    dic['White'] = data1[i][2]
    dic['Hispanic'] = data1[i][3]
    dic['Black'] = data1[i][4]
    dic['MedianAge'] = data1[i][5]
    
    # B07001~ Geographical Mobility in the Past Year by Age for Current Residence (int)
    dic['SameHouse'] = data1[i][6]
    dic['FromOtherCounty'] = data1[i][7]
    dic['FromOtherState'] = data1[i][8]
    dic['ToOtherCounty'] = data1[i][9]
    dic['ToOtherState'] = data1[i][10]
    
    # B07401!: Educational background (int)
    dic['HighSchool'] = data1[i][11]
    dic['Bachelor'] = data1[i][12]
    
    # BB06001~: Population characteristics (int)
    dic['18to24'] = data1[i][13]
    dic['65to74'] = data1[i][14]
    dic['BornInState'] = data1[i][15]
    
    # B06011 Median Income (int)
    dic['MedianIncome'] = data1[i][16] 
    
    # B25003~: Home ownership status (int)
    dic['Tenure'] = data1[i][17]
    dic['Owner'] = data1[i][18]
    
    # B22005~: Recipients of Foodstamp (int)
    dic['Foodstamp'] = data1[i][19]
    
    # B27011~: Employment status (int)
    dic['Labor'] = data1[i][20]
    dic['Unemployed'] = data1[i][21]
    dic['NotInLabor'] = data1[i][22]
    
    # C24050 Numbers employed in each sector (int)
    dic['Manufacture'] = data1[i][23]
    dic['Agriculture'] = data1[i][24]
    dic['Management'] = data1[i][25]
    
    # ST: State level FIPS code (int)
    dic['state_fips'] = data1[i][26]
    
    # COUNTY: County level FIPS code (int)
    dic['CountyFIPS'] = data1[i][27]
    
    # Append the dictionary "formatted" in each loop
    formatted.append(dic)

    


In [8]:
# Create some new variables by looping through the new list "formatted" 

for i in range(len(formatted)):
    
    # Split NAME variable (County, State) into two parts, using split function (str)
    formatted[i]['county_name'] = formatted[i]['CountyState'].split(',')[0]
    formatted[i]['State'] = formatted[i]['CountyState'].split(',')[1][1:]
    
    # Create the ratio of each major race from the demography data (float)
    formatted[i]['white_r'] = int(formatted[i]['White'])/int(formatted[i]['Population'])
    formatted[i]['black_r'] = int(formatted[i]['Black'])/int(formatted[i]['Population'])
    formatted[i]['hisp_r'] = int(formatted[i]['Hispanic'])/int(formatted[i]['Population'])
    
    # Create sectoral employment ratio from sector employmne data and employment status data (float)
    formatted[i]['agriculture_r'] = int(formatted[i]['Agriculture'])/int(formatted[i]['Labor'])
    formatted[i]['manufacture_r'] = int(formatted[i]['Manufacture'])/int(formatted[i]['Labor'])
    formatted[i]['management_r'] = int(formatted[i]['Management'])/int(formatted[i]['Labor'])
    
    # Create unemployment rate and activity rate from employment status data (float)
    formatted[i]['active_r'] = int(formatted[i]['Labor'])/int(formatted[i]['Population'])
    formatted[i]['unemp_r'] = int(formatted[i]['Unemployed'])/int(formatted[i]['Population'])
    
    # Create education related data from education status data and demography data (float)
    formatted[i]['highschool_r'] = int(formatted[i]['HighSchool'])/int(formatted[i]['Population'])
    formatted[i]['univ_r'] = int(formatted[i]['Bachelor'])/int(formatted[i]['Population'])
    
    # Create homeownership ratio from homeownership status data (float)
    formatted[i]['homeowner_r'] = int(formatted[i]['Owner'])/int(formatted[i]['Tenure'])
   

In [9]:
# Test 
print(formatted[129])
print(formatted[277])
print(formatted[321])

{'ToOtherState': '133', 'Owner': '4372', 'manufacture_r': 0.12373802743981362, 'active_r': 0.43910201761864165, 'county_name': 'Cross County', 'state_fips': '05', 'highschool_r': 0.30986075589656153, 'Manufacture': '956', '65to74': '1693', 'White': '12995', 'MedianIncome': '19490', 'SameHouse': '15474', 'State': 'Arkansas', 'Management': '1640', 'Black': '3943', 'Unemployed': '698', 'Hispanic': '293', 'white_r': 0.738562091503268, 'MedianAge': '40.1', 'NotInLabor': '5329', 'CountyFIPS': '037', 'Bachelor': '1479', 'univ_r': 0.08405797101449275, 'homeowner_r': 0.6352804417320547, 'BornInState': '13104', 'FromOtherCounty': '466', 'Population': '17595', 'unemp_r': 0.039670360897982385, 'management_r': 0.21227025627750454, 'Foodstamp': '0', 'FromOtherState': '170', 'hisp_r': 0.016652458084683148, 'ToOtherCounty': '983', 'Tenure': '6882', 'Agriculture': '538', 'Labor': '7726', '18to24': '1341', 'agriculture_r': 0.06963499870566917, 'HighSchool': '5452', 'CountyState': 'Cross County, Arkansas

In [11]:
# Create a CSV file

import csv

# Retrieve the key from "formatted"
keys = formatted[1].keys()

# Write the csv with keys in the column and formatted data in rows
with open('census2014.csv', 'w') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(formatted)

In [13]:
# Repeat the same procedure in 2010 data 

key="***************************"

base_url="http://api.census.gov/data/2010/acs5?"

response_format=".json"

# Seletct the corresponding variables

search_params = {"key":key,
                 "get":"NAME,B01001_001E,B01001H_001E,B01001I_001E,B01001B_001E,B01002_001E,B07001_017E,B07001_049E,B07001_065E,B07401_049E,B07401_065E,B16010_015E,B16010_041E,B06001_004E,B06001_011E,B06001_013E,B06011_001E,B25003_001E,B25003_002E,B22005C_002E,C24050_004E,C24050_002E,C24050_015E,ST,COUNTY",
                 "for":"county:*","in":"state:*"}    

r = requests.get(base_url, params=search_params)

data=json.loads(r.text)

data1=data[1:]

# Use different variable names (variable + "10") so that it can be combined later

formatted = []
for i in range(len(data1)):
    dic={}
    dic['CountyState10'] = data1[i][0]
    dic['Population10'] = data1[i][1]
    dic['White10'] = data1[i][2]
    dic['Hispanic10'] = data1[i][3]
    dic['Black10'] = data1[i][4]
    dic['MedianAge10'] = data1[i][5]
    dic['SameHouse10'] = data1[i][6]
    dic['FromOtherCounty10'] = data1[i][7]
    dic['FromOtherState10'] = data1[i][8]
    dic['ToOtherCounty10'] = data1[i][9]
    dic['ToOtherState10'] = data1[i][10]
    dic['HighSchool10'] = data1[i][11]
    dic['Bachelor10'] = data1[i][12]
    dic['18to24_10'] = data1[i][13]
    dic['65to74_10'] = data1[i][14]
    dic['BornInState10'] = data1[i][15]
    dic['MedianIncome10'] = data1[i][16] 
    dic['Tenure10'] = data1[i][17]
    dic['Owner10'] = data1[i][18]
    dic['Foodstamp10'] = data1[i][19]
    dic['Manufacture10'] = data1[i][20]
    dic['Agriculture10'] = data1[i][21]
    dic['Management10'] = data1[i][22]
    dic['state_fips'] = data1[i][23]
    dic['CountyFIPS'] = data1[i][24]
    formatted.append(dic)

for i in range(len(formatted)):
    formatted[i]['county_name'] = formatted[i]['CountyState10'].split(',')[0]
    formatted[i]['State10'] = formatted[i]['CountyState10'].split(',')[1][1:]
    formatted[i]['white_r10'] = int(formatted[i]['White10'])/int(formatted[i]['Population10'])
    formatted[i]['black_r10'] = int(formatted[i]['Black10'])/int(formatted[i]['Population10'])
    formatted[i]['hisp_r10'] = int(formatted[i]['Hispanic10'])/int(formatted[i]['Population10'])
    formatted[i]['highschool_r10'] = int(formatted[i]['HighSchool10'])/int(formatted[i]['Population10'])
    formatted[i]['univ_r10'] = int(formatted[i]['Bachelor10'])/int(formatted[i]['Population10'])
    formatted[i]['homeowner_r10'] = int(formatted[i]['Owner10'])/int(formatted[i]['Tenure10'])
   
  
import csv
keys = formatted[1].keys()

#writing the csv file with slightly different name 
with open('census2010.csv', 'w') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(formatted)
    
print(formatted[129])
print(formatted[277])
print(formatted[321])

{'FromOtherCounty10': '353', 'ToOtherState10': '507', 'highschool_r10': 0.27992724867724866, '65to74_10': '1357', 'county_name': 'Cross County', 'state_fips': '05', 'HighSchool10': '5079', 'CountyFIPS': '037', 'Black10': '4164', 'Population10': '18144', 'black_r10': 0.2294973544973545, 'State10': 'Arkansas', 'BornInState10': '13386', 'Agriculture10': '579', 'Management10': '1939', 'MedianAge10': '38.4', 'Tenure10': '6638', 'hisp_r10': 0.013723544973544973, 'Bachelor10': '1506', 'SameHouse10': '15433', 'Hispanic10': '249', 'White10': '13478', 'ToOtherCounty10': '783', 'univ_r10': 0.08300264550264551, 'CountyState10': 'Cross County, Arkansas', 'Owner10': '4690', 'MedianIncome10': '19288', 'homeowner_r10': 0.7065381138897259, 'FromOtherState10': '312', 'Manufacture10': '770', 'white_r10': 0.7428350970017636, '18to24_10': '1473', 'Foodstamp10': '6'}
{'FromOtherCounty10': '450', 'ToOtherState10': '184', 'highschool_r10': 0.15229435999431737, '65to74_10': '309', 'county_name': 'Lake County',